<a href="https://colab.research.google.com/github/amylynnn/weatherwise-Amylynn-Sophie/blob/main/DRAFT%202%20Untitled19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ✅ Install packages
!pip install -q requests pandas matplotlib plotly pyinputplus

# ✅ Set API key for OpenWeatherMap
import os
os.environ['OPENWEATHER_API_KEY'] = input("🔐 Enter your OpenWeatherMap API Key: ")

# ✅ Imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import pyinputplus as pyip
import re
from typing import Union, List, Dict

# ✅ Weather Data Fetcher from OpenWeatherMap
def get_weather_data(location: Union[str, List[str]], forecast_days: int = 3) -> Dict[str, dict]:
    if isinstance(location, str):
        location = [location]
    results = {}
    api_key = os.environ.get('OPENWEATHER_API_KEY')
    for loc in location:
        try:
            url = f"http://api.openweathermap.org/data/2.5/forecast?q={loc}&appid={api_key}&units=metric"
            res = requests.get(url)
            if res.status_code != 200:
                raise ValueError(f"{res.status_code} - {res.text}")
            raw = res.json()
            forecast = {}
            for entry in raw["list"]:
                date = entry["dt_txt"].split()[0]
                if date not in forecast:
                    forecast[date] = {
                        "temperature": entry["main"]["temp"],
                        "rain": "rain" in entry.get("weather", [{}])[0]["description"].lower(),
                        "humidity": entry["main"]["humidity"],
                        "wind": entry["wind"]["speed"]
                    }
                    if len(forecast) >= forecast_days:
                        break
            results[loc] = {
                "current_conditions": raw["list"][0]["main"],
                "forecast": forecast
            }
        except Exception as e:
            results[loc] = {"error": f"❌ Failed to retrieve weather for {loc}: {e}"}
    return results

# ✅ Simple question parser (no AI needed)
def parse_weather_question(question):
    question = question.lower()
    loc_match = re.search(r'in ([a-zA-Z ]+)', question)
    location = loc_match.group(1).title().strip() if loc_match else 'your area'
    if 'tomorrow' in question:
        time = 'tomorrow'
    elif 'weekend' in question:
        time = 'this weekend'
    else:
        time = list(get_weather_data(location).get(location, {}).get("forecast", {}).keys())[0]  # fallback
    for attr in ['temperature', 'rain', 'snow', 'wind', 'humidity']:
        if attr in question:
            attribute = attr
            break
    else:
        attribute = 'temperature'
    return {'location': location, 'time_period': time, 'attribute': attribute}

# ✅ Unit conversion and response generation
def convert_temperature(value, unit='Celsius'):
    if unit == "Fahrenheit":
        return round((value * 9/5) + 32)
    return round(value)

def convert_wind_speed(value, unit='km/h'):
    if unit == "mph":
        return round(value / 1.609)
    return round(value)

def generate_weather_response(parsed, data, temp_unit='Celsius', wind_unit='km/h'):
    loc = parsed['location']
    time = parsed['time_period']
    attr = parsed['attribute']
    if loc not in data or time not in data[loc]:
        return f"❌ No forecast available for {loc} {time}."
    forecast = data[loc][time]
    def convert_temp(c): return f"{convert_temperature(c, temp_unit)}°{'F' if temp_unit == 'Fahrenheit' else 'C'}"
    def convert_wind(w): return f"{convert_wind_speed(w, wind_unit)} {wind_unit}"
    responses = {
        'temperature': lambda: f"🌡️ Temperature in {loc} {time} is {convert_temp(forecast.get('temperature', 0))}.",
        'rain': lambda: f"🌧️ Expect {'rain' if forecast.get('rain') else 'no rain'} in {loc} {time}.",
        'snow': lambda: f"❄️ Snow forecast not available, assuming none in {loc} {time}.",
        'wind': lambda: f"💨 Wind speed in {loc} {time} is {convert_wind(forecast.get('wind', 0))}.",
        'humidity': lambda: f"💧 Humidity in {loc} {time} is {forecast.get('humidity', 'unknown')}%."
    }
    return responses.get(attr, lambda: "Invalid attribute")()

# ✅ Visualisation functions
def create_temperature_visualisation_interactive(weather_data):
    rows = []
    for location, times_data in weather_data.items():
        for time, info in times_data.items():
            rows.append({"Location": location, "Time": time, "Temperature": info["temperature"]})
    df = pd.DataFrame(rows)
    fig = px.line(df, x="Time", y="Temperature", color="Location", markers=True,
                  title="Interactive Temperature Forecast")
    fig.update_traces(hovertemplate="%{y}°C at %{x}<br>%{fullData.name}")
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

# ✅ Menu and interface
def print_banner():
    print("\n" + "="*40)
    print("☁️  Welcome to the Weather Assistant!  ☁️")
    print("="*40)

def main_menu():
    return pyip.inputMenu(
        ["Ask Weather Question", "Visualize Temperature", "Exit"],
        lettered=True,
        numbered=False,
        prompt="\n🌈 What would you like to do?\n"
    )

def run_interface():
    print_banner()
    while True:
        choice = main_menu()
        if "Ask" in choice:
            user_question = input("🗣️ Ask a weather question: ")
            parsed = parse_weather_question(user_question)
            data = get_weather_data(parsed['location'])
            structured = {
                parsed['location']: {
                    parsed['time_period']: data.get(parsed['location'], {}).get('forecast', {}).get(parsed['time_period'], {})
                }
            }
            print(generate_weather_response(parsed, structured))
        elif "Visualize" in choice:
            city = input("📍 Enter city for temperature chart: ")
            data = get_weather_data(city)
            if 'error' in data.get(city, {}):
                print(data[city]['error'])
            else:
                create_temperature_visualisation_interactive({city: data[city]["forecast"]})
        elif "Exit" in choice:
            print("👋 Goodbye!")
            break

# ▶️ Start
run_interface()


🔐 Enter your OpenWeatherMap API Key: 7cf20335110caaf78db0fecb31852d45

☁️  Welcome to the Weather Assistant!  ☁️

🌈 What would you like to do?
A. Ask Weather Question
B. Visualize Temperature
C. Exit
a
🗣️ Ask a weather question: Whats the weather today in paris
🌡️ Temperature in Paris 2025-05-20 is 23°C.

🌈 What would you like to do?
A. Ask Weather Question
B. Visualize Temperature
C. Exit
b
📍 Enter city for temperature chart: paris



🌈 What would you like to do?
A. Ask Weather Question
B. Visualize Temperature
C. Exit
c
👋 Goodbye!
